In [4]:
# !pip install pydot
from sklearn.pipeline import Pipeline
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.wrappers.scikit_learn import KerasRegressor
import pandas as pd
import numpy as np
from datetime import date
from keras.preprocessing.sequence import TimeseriesGenerator
import seaborn as sns
import matplotlib.pyplot as plt
import keras
import pydot
from keras.losses import MeanAbsolutePercentageError
from keras import Input

In [21]:
def load_ts_data(stock_name='GSIT'):
    stocks_df = pd.read_pickle('../data/stocks_df_{}.pickle'.format(date.today()))
    input_data = np.array(stocks_df[stocks_df.company_name == stock_name].iloc[:-60]['Adj Close'])
    targets = np.array(stocks_df[stocks_df.company_name == stock_name].iloc[60:]['Adj Close'])
    ts_generator = TimeseriesGenerator(
        input_data, targets, length=1)
    return ts_generator
load_ts_data()

In [22]:
def plot_stock_close(stock_name='GSIT'):
    try:
        stocks_df = pd.read_pickle('../data/stocks_df_{}.pickle'.format(date.today()))
    except:
        print('rerun stock_data_ingestion.py to get up-to-date data')
    stock_ts = stocks_df[stocks_df.company_name == stock_name]['Adj Close']
    # ax = sns.scatterplot(data=stock_ts)
    ax = sns.lineplot(data=stock_ts)
    plt.show()

In [23]:
# plot_stock_close()

In [24]:
def run(stock_name='GSIT'):
    ts_data = load_ts_data(stock_name)
    return ts_data
ts_data = run()
print(len(ts_data))
for batch in ts_data:
    indata, target = batch
    print(indata.shape, target.shape)

10
(128, 1) (128,)
(128, 1) (128,)
(128, 1) (128,)
(128, 1) (128,)
(128, 1) (128,)
(128, 1) (128,)
(128, 1) (128,)
(128, 1) (128,)
(128, 1) (128,)
(47, 1) (47,)


In [36]:
def make_model(input_shape=(128, 1)):

    input_layer = keras.layers.Input(input_shape)
    conv1 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(input_layer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.ReLU()(conv1)

    conv2 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.ReLU()(conv2)

    conv3 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.ReLU()(conv3)

    gap = keras.layers.GlobalAveragePooling1D()(conv3)

    output_layer = keras.layers.Dense(1, activation="softmax")(gap)

    return keras.models.Model(inputs=input_layer, outputs=output_layer)

def make_sequential(input_shape=(128, 60)):
    model = Sequential(
        [
#             Input(shape=input_shape),
            Dense(2, activation="relu", name="layer1"),
            Dense(3, activation="relu", name="layer2"),
            Dense(4, name="layer3"),
        ])
    return model


In [39]:
model = make_sequential()
# model.summary()

In [42]:
ts_generator = load_ts_data()
# model = makesequential(input_shape=(128, 1))
# keras.utils.plot_model(model, show_shapes=True)
model.compile(optimizer='adam', loss=MeanAbsolutePercentageError())
for batch in ts_generator:
#     print(batch[0].shape, batch[1].shape)
#     print(batch[0])
#     break
    model.fit(batch[0])

ValueError: in user code:

    /Users/admin/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /Users/admin/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/admin/miniconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/admin/miniconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/admin/miniconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    /Users/admin/miniconda3/lib/python3.8/site-packages/tensorf

In [12]:
print(ts_generator.__len__())
for x in ts_generator:
    print(x)

10
(array([[[3.45000005]],

       [[3.41000009]],

       [[3.4000001 ]],

       [[3.44000006]],

       [[3.45000005]],

       [[3.79999995]],

       [[3.91000009]],

       [[3.94000006]],

       [[3.88000011]],

       [[3.97000003]],

       [[4.01999998]],

       [[4.03999996]],

       [[3.88000011]],

       [[3.8900001 ]],

       [[3.79999995]],

       [[3.8900001 ]],

       [[3.94000006]],

       [[3.91000009]],

       [[3.99000001]],

       [[4.01000023]],

       [[4.38999987]],

       [[4.38000011]],

       [[4.38000011]],

       [[4.09000015]],

       [[4.28999996]],

       [[4.11999989]],

       [[4.05000019]],

       [[4.13000011]],

       [[4.0999999 ]],

       [[4.15999985]],

       [[4.05000019]],

       [[4.05999994]],

       [[3.92000008]],

       [[3.8599999 ]],

       [[3.80999994]],

       [[3.91000009]],

       [[4.0999999 ]],

       [[4.11000013]],

       [[4.19999981]],

       [[4.        ]],

       [[3.96000004]],

       [[4.0